# Importing libraries

In [1]:
import sys, os, re, csv, subprocess, operator
import pandas as pd
from urllib.request import urlopen
import urllib.request
from pprint import pprint
from bs4 import BeautifulSoup

# Defining repository and directories

In [2]:
userhome = os.path.expanduser("~")
repository = userhome + r"/different-diff/dataset/hbase/"
analyze_dir = userhome + r'/different-diff/analyze/analyze_hbase/'

# Load textfile contains bug-ids

In [3]:
txt_file = open(analyze_dir + "01_code_document/txt_files/candidate_bug_links.txt", "r")
bug_links = txt_file.read().split('\n')
print ("Found " + str(len(bug_links)) + " bug_ids")

Found 624 bug_ids


In [4]:
print (bug_links)

['HBASE-9971', 'HBASE-9961', 'HBASE-9958', 'HBASE-9908', 'HBASE-9893', 'HBASE-9856', 'HBASE-985', 'HBASE-9843', 'HBASE-984', 'HBASE-9786', 'HBASE-977', 'HBASE-9747', 'HBASE-9745', 'HBASE-9704', 'HBASE-9698', 'HBASE-968', 'HBASE-9639', 'HBASE-9534', 'HBASE-9519', 'HBASE-9415', 'HBASE-930', 'HBASE-927', 'HBASE-9238', 'HBASE-9232', 'HBASE-9219', 'HBASE-9205', 'HBASE-9164', 'HBASE-9158', 'HBASE-9151', 'HBASE-9115', 'HBASE-9114', 'HBASE-9085', 'HBASE-9003', 'HBASE-9001', 'HBASE-8942', 'HBASE-8930', 'HBASE-8919', 'HBASE-8917', 'HBASE-8856', 'HBASE-881', 'HBASE-8798', 'HBASE-8706', 'HBASE-8664', 'HBASE-8627', 'HBASE-8615', 'HBASE-8532', 'HBASE-8521', 'HBASE-8519', 'HBASE-8500', 'HBASE-8467', 'HBASE-830', 'HBASE-826', 'HBASE-8259', 'HBASE-8225', 'HBASE-8223', 'HBASE-8216', 'HBASE-8207', 'HBASE-8206', 'HBASE-8125', 'HBASE-8119', 'HBASE-8111', 'HBASE-811', 'HBASE-8085', 'HBASE-808', 'HBASE-8067', 'HBASE-8062', 'HBASE-8058', 'HBASE-8050', 'HBASE-8044', 'HBASE-7966', 'HBASE-7953', 'HBASE-7920', 'H

# Finding affected versions by bug links

In [5]:
error_links = []
affected_version = []
for a,b in enumerate(bug_links):
    link = "https://issues.apache.org/jira/browse/" + b
    sys.stdout.write("\r%i " %(a+1) + "Extracting: " + b)
    sys.stdout.flush()
    try:
        page = urllib.request.urlopen(link)
        soup = BeautifulSoup(page, 'html.parser')
        aff_version = soup.find('span', attrs={'id':'versions-val'}).text.replace("\n",'').replace(" ",'').split(",")
        aff_version = sorted(aff_version)
        aff_version.insert(0, b)
        affected_version.append(aff_version)
    except:
        error_links.append(b)

print("\nExtraction has been completed.")

624 Extracting: HBASE-15195
Extraction has been completed.


In [6]:
print (error_links)

[]


In [12]:
#Repeat the process if there are still some affected versions by bug_ids haven't been captured
errorlinks = []
if error_links != []:
    for c,d in enumerate(error_links):
        link = "https://issues.apache.org/jira/browse/" + d
        sys.stdout.write("\r%i " %(c+1) + "Extracting: " + d)
        sys.stdout.flush()
        try:
            page = urllib.request.urlopen(link)
            soup = BeautifulSoup(page, 'html.parser')
            types = soup.find('span', attrs={'id':'versions-val'}).text.replace("\n",'').replace(" ",'').split(",")
            types = sorted(types)
            types.insert(0, d)
            affected_version.append(types)
        except:
            errorlinks.append(b)

print ("\nExtraction is complete")


Extraction is complete


In [13]:
print (errorlinks)

[]


In [14]:
affected_version

[['HBASE-9971', 'None'],
 ['HBASE-9961', 'None'],
 ['HBASE-9958', '0.96.0', '0.98.0'],
 ['HBASE-9908', 'None'],
 ['HBASE-9893', '0.96.0'],
 ['HBASE-9856', 'None'],
 ['HBASE-985', '0.19.0'],
 ['HBASE-9843', '0.96.0'],
 ['HBASE-984', '0.19.0'],
 ['HBASE-9786', '0.96.0', '0.98.0'],
 ['HBASE-977', 'None'],
 ['HBASE-9747', '0.94.9', '0.95.2'],
 ['HBASE-9745', '0.94.11', '0.95.2', '0.98.0'],
 ['HBASE-9704', '0.89-fb'],
 ['HBASE-9698', 'None'],
 ['HBASE-968', 'None'],
 ['HBASE-9639', '0.95.2'],
 ['HBASE-9534', 'None'],
 ['HBASE-9519', '0.96.1', '0.98.0'],
 ['HBASE-9415', '0.96.0'],
 ['HBASE-930', 'None'],
 ['HBASE-927', 'None'],
 ['HBASE-9238', 'None'],
 ['HBASE-9232', 'None'],
 ['HBASE-9219', '0.89-fb'],
 ['HBASE-9205', '0.95.1'],
 ['HBASE-9164', '0.95.1'],
 ['HBASE-9158', '0.94.10', '0.95.1', '0.98.0'],
 ['HBASE-9151', 'None'],
 ['HBASE-9115', '0.94.10'],
 ['HBASE-9114', '0.89-fb'],
 ['HBASE-9085', '0.94.10', '0.94.9', '0.95.0'],
 ['HBASE-9003', 'None'],
 ['HBASE-9001', 'None'],
 ['HBASE-89

In [15]:
#Finding the earliest version affected by the bug ids
earliest_version = []
for num, affver in enumerate(affected_version):
    earliest_version.append(affver[:2])

earliest_version.sort()
for early in earliest_version:
    print (early)

['HBASE-10004', '0.98.0']
['HBASE-10011', '0.96.0']
['HBASE-10079', 'None']
['HBASE-1013', 'None']
['HBASE-10370', '0.94.3']
['HBASE-10397', '0.99.0']
['HBASE-10417', 'None']
['HBASE-10446', 'None']
['HBASE-10452', '0.96.1']
['HBASE-10454', '0.98.0']
['HBASE-10460', '0.98.0']
['HBASE-10485', 'None']
['HBASE-10514', 'None']
['HBASE-10528', '0.94.5']
['HBASE-10555', '0.94.16']
['HBASE-10578', '0.89-fb']
['HBASE-1058', 'None']
['HBASE-10598', '0.94.16']
['HBASE-10624', 'None']
['HBASE-10647', '0.89-fb']
['HBASE-10656', 'None']
['HBASE-10675', 'None']
['HBASE-10715', 'None']
['HBASE-10808', '0.89-fb']
['HBASE-10840', 'None']
['HBASE-10893', '0.89-fb']
['HBASE-10901', '0.89-fb']
['HBASE-10907', '0.89-fb']
['HBASE-1092', 'None']
['HBASE-11021', '0.89-fb']
['HBASE-11120', '0.98.2']
['HBASE-11138', '0.89-fb']
['HBASE-11145', 'None']
['HBASE-11149', 'None']
['HBASE-11169', '0.95.0']
['HBASE-11187', '0.89-fb']
['HBASE-1121', 'None']
['HBASE-11217', 'None']
['HBASE-11225', 'None']
['HBASE-11234',

['HBASE-9856', 'None']
['HBASE-9893', '0.96.0']
['HBASE-9908', 'None']
['HBASE-9958', '0.96.0']
['HBASE-9961', 'None']
['HBASE-9971', 'None']


# Defining the function for git command

In [16]:
def execute_command(cmd, work_dir):
    #Executes a shell command in a subprocess, waiting until it has completed.
    pipe = subprocess.Popen(cmd, shell=True, cwd=work_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    (out, error) = pipe.communicate()
    return out, error
    pipe.wait()

# Finding the versions related with earliest version

In [17]:
related_version = []
for n, item in enumerate(earliest_version):
    if item[1] != "None":
        git_cmd = 'git tag -l "*' + item[1] + '*"'
        temp = str(execute_command(git_cmd, repository)).replace("b'",'').replace("(",'').replace(")",'').split("\\n")
        del temp[len(temp)-1]
        if temp == []:
            temp = [item[1]]
    else:
        temp = ['None']

    temp.insert(0, item[0])
    related_version.append(temp)

for xx in related_version:
    print (xx)

['HBASE-10004', '0.98.0', '0.98.0RC0', '0.98.0RC1', '0.98.0RC2', 'rel/0.98.0']
['HBASE-10011', '0.96.0RC0', '0.96.0RC1', '0.96.0RC3', '0.96.0RC4', '0.96.0RC5', '0.96.0rc2', 'hbase-0.96.0', 'rel/0.96.0']
['HBASE-10079', 'None']
['HBASE-1013', 'None']
['HBASE-10370', '0.94.3', '0.94.3RC0', '0.94.3mvn', '0.94.3mvn2', '0.94.3mvn_release', 'rel/0.94.3']
['HBASE-10397', '0.99.0', '0.99.0RC0', '0.99.0RC1', 'rel/0.99.0']
['HBASE-10417', 'None']
['HBASE-10446', 'None']
['HBASE-10452', '0.96.1', '0.96.1.1', '0.96.1.1rc0', '0.96.1RC0', '0.96.1RC2', '0.96.1new', 'hbase-0.96.1RC1', 'rel/0.96.1', 'rel/0.96.1.1']
['HBASE-10454', '0.98.0', '0.98.0RC0', '0.98.0RC1', '0.98.0RC2', 'rel/0.98.0']
['HBASE-10460', '0.98.0', '0.98.0RC0', '0.98.0RC1', '0.98.0RC2', 'rel/0.98.0']
['HBASE-10485', 'None']
['HBASE-10514', 'None']
['HBASE-10528', '0.94.5', '0.94.5RC0', '0.94.5RC1', '0.94.5mvn', '0.94.5mvn2', 'rel/0.94.5']
['HBASE-10555', '0.94.16', '0.94.16RC0', '0.94.16mvn', '0.94.16mvn2', 'rel/0.94.16']
['HBASE-10

# Finding the date release for affected version

In [18]:
related_version[1][1]

'0.96.0RC0'

In [19]:
date_release = []
for n, item in enumerate(related_version):
    sys.stdout.write("\rFinding datetime for version {}: {}".format(n+1, item[0]))
    sys.stdout.flush()
    if item[1] != "None":
        for m in range(1, len(item)):
            git_cmd = "git log -1 --format=%ai " + item[m]
            temp = str(execute_command(git_cmd, repository)).replace("b'",'').replace("(",'').replace(")",'').split("\\n")
            del temp[len(temp)-1]
            temp = temp[0].split(" ")
            if temp[0] != "',":
                temp.insert(0,item[0])
                temp.insert(1,item[m])
                date_release.append(temp)
                date_release = sorted(date_release, key=operator.itemgetter(0, 2))
    """else:
        date_release.append(item)"""

date_release = sorted(date_release, key=operator.itemgetter(0), reverse=True) 

print("\n")
for jj in date_release:
    print (jj)

Finding datetime for version 624: HBASE-99719

['HBASE-9958', '0.96.0RC0', '2013-08-31', '20:58:53', '+0000']
['HBASE-9958', '0.96.0RC1', '2013-09-17', '08:10:28', '+0000']
['HBASE-9958', '0.96.0rc2', '2013-09-20', '04:09:13', '+0000']
['HBASE-9958', '0.96.0RC3', '2013-09-21', '22:54:29', '+0000']
['HBASE-9958', '0.96.0RC4', '2013-10-05', '00:17:47', '+0000']
['HBASE-9958', '0.96.0RC5', '2013-10-11', '22:00:07', '+0000']
['HBASE-9958', 'hbase-0.96.0', '2013-10-19', '15:52:30', '+0000']
['HBASE-9958', 'rel/0.96.0', '2013-10-19', '15:52:30', '+0000']
['HBASE-9893', '0.96.0RC0', '2013-08-31', '20:58:53', '+0000']
['HBASE-9893', '0.96.0RC1', '2013-09-17', '08:10:28', '+0000']
['HBASE-9893', '0.96.0rc2', '2013-09-20', '04:09:13', '+0000']
['HBASE-9893', '0.96.0RC3', '2013-09-21', '22:54:29', '+0000']
['HBASE-9893', '0.96.0RC4', '2013-10-05', '00:17:47', '+0000']
['HBASE-9893', '0.96.0RC5', '2013-10-11', '22:00:07', '+0000']
['HBASE-9893', 'hbase-0.96.0', '2013-10-19', '15:52:30', '+0000']
[

['HBASE-8062', '0.95.0', '2013-04-07', '22:23:49', '+0000']
['HBASE-8062', '0.95.0mvn-release', '2013-04-07', '23:45:54', '+0000']
['HBASE-8062', '0.95.0mvnrelease', '2013-04-07', '22:29:53', '+0000']
['HBASE-8062', 'hbase-0.95.0', '2013-04-07', '22:02:24', '+0000']
['HBASE-8062', 'rel/0.95.0', '2013-04-07', '22:23:49', '+0000']
['HBASE-8062', '0.95.0mvn', '2013-04-08', '00:08:41', '+0000']
['HBASE-8044', '0.94.5RC0', '2013-02-02', '07:39:53', '+0000']
['HBASE-8044', '0.94.5RC1', '2013-02-08', '05:51:57', '+0000']
['HBASE-8044', '0.94.5', '2013-02-20', '05:43:22', '+0000']
['HBASE-8044', 'rel/0.94.5', '2013-02-20', '05:43:22', '+0000']
['HBASE-8044', '0.94.5mvn', '2013-02-21', '20:02:09', '+0000']
['HBASE-8044', '0.94.5mvn2', '2013-02-21', '20:02:06', '+0000']
['HBASE-7920', '0.94.5RC0', '2013-02-02', '07:39:53', '+0000']
['HBASE-7920', '0.94.5RC1', '2013-02-08', '05:51:57', '+0000']
['HBASE-7920', '0.94.5', '2013-02-20', '05:43:22', '+0000']
['HBASE-7920', 'rel/0.94.5', '2013-02-20', 

['HBASE-6377', '0.94.16', '2014-01-20', '21:55:46', '+0000']
['HBASE-6377', 'rel/0.94.16', '2014-01-20', '21:55:46', '+0000']
['HBASE-6377', '0.94.16mvn', '2014-01-21', '01:23:45', '+0000']
['HBASE-6377', '0.94.16mvn2', '2014-01-21', '01:23:43', '+0000']
['HBASE-6377', '0.94.17RC0', '2014-02-13', '22:59:32', '+0000']
['HBASE-6377', '0.94.17RC1', '2014-02-18', '19:46:32', '+0000']
['HBASE-6377', '0.94.17', '2014-02-26', '04:17:25', '+0000']
['HBASE-6377', '0.94.17mvn', '2014-02-26', '04:24:21', '+0000']
['HBASE-6377', '0.94.17mvn2', '2014-02-26', '04:24:20', '+0000']
['HBASE-6377', 'rel/0.94.17', '2014-02-26', '04:17:25', '+0000']
['HBASE-6377', '0.94.18RC0', '2014-03-15', '04:41:33', '+0000']
['HBASE-6377', '0.94.18', '2014-03-24', '18:50:53', '+0000']
['HBASE-6377', '0.94.18mvn', '2014-03-24', '22:41:06', '+0000']
['HBASE-6377', '0.94.18mvn2', '2014-03-24', '22:41:04', '+0000']
['HBASE-6377', 'rel/0.94.18', '2014-03-24', '18:50:53', '+0000']
['HBASE-6377', '0.94.19RC0', '2014-04-21', 

['HBASE-16066', 'rel/2.0.0-alpha-1', '2017-06-07', '14:29:19', '-0500']
['HBASE-16066', '2.0.0-alpha-2RC0', '2017-08-02', '22:54:21', '-0700']
['HBASE-16066', 'rel/2.0.0-alpha-2', '2017-08-02', '22:54:21', '-0700']
['HBASE-16066', '2.0.0-alpha-3RC0', '2017-09-14', '17:20:59', '-0700']
['HBASE-16066', '2.0.0-alpha-3RC0.2', '2017-09-15', '16:35:21', '+0800']
['HBASE-16066', 'rel/2.0.0-alpha-3', '2017-09-15', '16:35:21', '+0800']
['HBASE-16066', '2.0.0-alpha4RC0', '2017-10-31', '10:08:53', '-0700']
['HBASE-16066', '2.0.0-beta-1-RC0', '2017-12-28', '22:44:06', '+0200']
['HBASE-16066', '2.0.0-beta-1-RC0.2', '2017-12-29', '08:27:08', '-0800']
['HBASE-16066', '2.0.0-beta-1-RC1.2', '2017-12-29', '08:27:08', '-0800']
['HBASE-16066', '2.0.0-beta-1-RC1', '2018-01-05', '08:31:07', '-0600']
['HBASE-16066', '2.0.0-beta-1-RC1.3', '2018-01-05', '18:13:54', '-0500']
['HBASE-16066', '2.0.0-beta-1-RC1.4', '2018-01-06', '08:51:19', '-0600']
['HBASE-16066', '2.0.0-beta-1-RC1.5', '2018-01-08', '16:18:30', '

In [21]:
#save in CSV file
with open(analyze_dir + '05_finding_versions/01_affected_version/affected_version.csv','w') as csvfile:
    writers = csv.writer(csvfile)
    writers.writerow(['bug_id','earliest_affected_version','date_release','time_release','tz'])
    for item in date_release:
        writers.writerow(item)

In [22]:
df = pd.read_csv(analyze_dir + '05_finding_versions/01_affected_version/affected_version.csv')
df

,bug_id,earliest_affected_version,date_release,time_release,tz
0,HBASE-9958,0.96.0RC0,2013-08-31,20:58:53,0
1,HBASE-9958,0.96.0RC1,2013-09-17,08:10:28,0
2,HBASE-9958,0.96.0rc2,2013-09-20,04:09:13,0
3,HBASE-9958,0.96.0RC3,2013-09-21,22:54:29,0
4,HBASE-9958,0.96.0RC4,2013-10-05,00:17:47,0
5,HBASE-9958,0.96.0RC5,2013-10-11,22:00:07,0
6,HBASE-9958,hbase-0.96.0,2013-10-19,15:52:30,0
7,HBASE-9958,rel/0.96.0,2013-10-19,15:52:30,0
8,HBASE-9893,0.96.0RC0,2013-08-31,20:58:53,0
9,HBASE-9893,0.96.0RC1,2013-09-17,08:10:28,0


In [24]:
earliest_vers = df.groupby('bug_id', as_index=False).first()
earliest_vers = earliest_vers.sort_values(['date_release', 'time_release', 'earliest_affected_version'], ascending=True)
earliest_vers.to_csv(analyze_dir + '05_finding_versions/01_affected_version/earliest_version.csv')
earliest_vers

,bug_id,earliest_affected_version,date_release,time_release,tz
188,HBASE-500,0.1.0,2010-05-18,20:49:38,0
189,HBASE-501,0.1.0,2010-05-18,20:49:38,0
190,HBASE-505,0.1.0,2010-05-18,20:49:38,0
198,HBASE-552,0.1.0,2010-05-18,20:49:38,0
199,HBASE-554,0.1.0,2010-05-18,20:49:38,0
209,HBASE-613,0.1.0,2010-05-18,20:49:38,0
195,HBASE-532,0.1.1,2010-05-18,20:49:38,0
221,HBASE-681,0.1.2,2010-05-18,20:49:38,0
256,HBASE-881,0.18.0,2010-05-18,20:49:38,0
14,HBASE-1155,0.19.0,2010-05-18,20:49:38,0


# Finding fix versions of bug links

In [25]:
errors = []
fixed_version = []
for a,b in enumerate(bug_links):
    link = "https://issues.apache.org/jira/browse/" + b
    sys.stdout.write("\r%i " % (a+1) + "Extracting: " + b)
    sys.stdout.flush()
    try:
        page = urllib.request.urlopen(link)
        soup = BeautifulSoup(page, 'html.parser')
        fix_version = soup.find('span', attrs={'id':'fixfor-val'}).text.replace("\n",'').replace(" ",'').split(",")
        fix_version = sorted(fix_version)
        fix_version.insert(0, b)
        fixed_version.append(fix_version)
    except:
        errors.append(b)

print("\nExtraction has been completed.")

624 Extracting: HBASE-15195
Extraction has been completed.


In [ ]:
errors

In [ ]:
for itemfix in fixed_version:
    print (itemfix)